![AES_Masthead](https://kyrill.ias.sdsmt.edu/wjc/eduresources/AES_Masthead.png)
# WRF Standalone Script for Plotting SkewT's from WRFOUT Files.

---
## Overview
Script makes professional quality detailed Skew-Ts including a hodograph and humidity windows as well as key thermodynamic and storm indicies.

Product Example: 
![SDSMT Realtime WRF Image](https://kyrill.ias.sdsmt.edu/wjc/eduresources/wrfout_d01_2024-10-10_0000_SKEWT_KRAP.gif)

---
## Data Requirements and User Modification Needs

This program requires the following files and data

*  A single wrf_out file. (can be in original or renamed format)
    * e.g., "*./wrfout_d01_2024-04-03_00:00:00*" or "*./wrfout_d01_2024-04-03_00.nc*"
    * File can be read locally or through an OPeNDAP/DODS/THREDDS service.
*  A WRF "*tslist*" file (even if you don't use it in WRF to make timeseries files)
    *   Strict Column Formatting is Required:
        *   Fortran-style format =**'(A25,1X,A5,1X,F7.3,1X,F8.3)'**
            *   Full Station Name (25 characters, whitespace clipped*)
            *   Station Abbrev. (for saving the skewt files) (5 characters, WSC*)
            *   Latitude in degrees north (7 digits long)
            *   Longitude in degrees east (8 digits long)
    *   Example File Available at
        *   [https://kyrill.ias.sdsmt.edu/wjc/eduresources/tslist](https://kyrill.ias.sdsmt.edu/wjc/eduresources/tslist)
    *   File can be read locally or through an HTTPS service.
*  Writable directory space to store the Skew_T Files
    *   The directory should be local relative to code.
 
*  This code uses iPython such as magic commands for logging [version information](https://nbviewer.org/github/jrjohansson/version_information/blob/master/example.ipynb) and the [display](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html) modules. Saving this as a standalone script will require commenting out the *version_information* line under the Library Loading Section and changing the *display* modules to the more basic and not-as-pretty *print* command.

## Post Processing Capability

There is room in the code (currently commented off) to group the resulting PNG files into a single animated GIF using [Imagemagik's](https://imagemagick.org/index.php) *convert* utility.

---
## Libraries

The following library dependancies are required.

<div class="alert alert-block alert-warning">As of 22 November 2024, WRF Python still requires Python 3.11 and may require a separate enviornment to accomodate the older generation of Python.  Instructions can be found here courtesy of Eric Salathé : <a href="https://groups.google.com/a/ucar.edu/g/wrfpython-talk/c/QfZ_eEfTbDI">https://groups.google.com/a/ucar.edu/g/wrfpython-talk/c/QfZ_eEfTbDI</a>
</div>

**Internal Libraries**

* [os](https://docs.python.org/3/library/os.html): Miscellaneous operating system interfaces
* [datetime](https://docs.python.org/3/library/datetime.html): Basic date and time types

**External Libraries Available Through Conda-Forge**

* [numpy](https://numpy.org/doc/stable): provides a multidimensional array object, various derived objects (such as masked arrays and matrices), and an assortment of routines for fast operations on arrays
* [xarray](https://docs.xarray.dev/en/stable/): library suppot multi-dimensional (a.k.a. N-dimensional, ND) arrays for self-describing files
* [pandas](https://pandas.pydata.org/docs/index.html): high-performance, easy-to-use data structures and data analysis tools for the Python programming language
* [scipy.interpolate](https://docs.scipy.org/doc/scipy/reference/interpolate.html): A subpackage for Scientific Python for objects used in interpolation
* [matplotlib.*](https://matplotlib.org/stable/): library for creating static, animated, and interactive visualizations
* [pint_xarray](https://pint-xarray.readthedocs.io/en/stable/): units support for xarray.
* [pint_pandas](https://pint-pandas.readthedocs.io/en/latest/): units support for xarray.
* [netCDF4](https://unidata.github.io/netcdf4-python/): a Python interface to the netCDF C library
* [wrfpy](https://wrf-python.readthedocs.io/en/latest/): diagnostic and interpolation routines for use with output from the Weather Research and Forecasting (WRF-ARW) Model.
* [metpy](https://unidata.github.io/MetPy/latest/index.html): package for reading, visualizing, and performing calculations with weather data
* [timezonefinder](https://timezonefinder.readthedocs.io/en/stable/) : a fast and lightweight python package for looking up the corresponding timezone for given coordinates on earth entirely offline.
* [pytz](https://pythonhosted.org/pytz/): World timezone definitions, modern and historical.
* [version_information](https://nbviewer.org/github/jrjohansson/version_information/blob/master/example.ipynb): Robert Johansson IPython magic extension for showing version information for dependency modules in a notebook


In [1]:
####################################################
####################################################
####################################################
#
# Libraries
#

import os                      as     os

import datetime                as     datetime
from   datetime                import timezone

import numpy                   as     np

import xarray                  as     xr
import pandas                  as     pd

import scipy.interpolate       as     scintp

import matplotlib              as     mpl
import matplotlib.pyplot       as     plt
import matplotlib.patches      as     patches
import matplotlib.font_manager as     fm
import matplotlib.gridspec     as     gridspec
from   matplotlib.ticker       import (MultipleLocator, NullFormatter, ScalarFormatter)

import metpy.calc              as     mpcalc
from   metpy.plots             import SkewT, Hodograph
from   metpy.units             import units, pandas_dataframe_to_unit_arrays

import pint                    as pint
import pint_xarray             as px
import pint_pandas             as pp

import netCDF4                 as nc4

import wrf                     as wrf

import pytz                    as pytz

import timezonefinder          as tzf

%load_ext version_information
%version_information numpy, scipy, xarray, pandas, matplotlib, pint, pint_xarray, pint_pandas, metpy, netCDF4, wrf, pytz, timezonefinder

#
####################################################
####################################################
####################################################

Software,Version
Python,3.11.9 64bit [Clang 16.0.6 ]
IPython,8.26.0
OS,macOS 15.1.1 arm64 arm 64bit
numpy,1.26.4
scipy,1.14.0
xarray,2024.6.0
pandas,2.2.2
matplotlib,3.8.4
pint,0.24.3
pint_xarray,0.4


---
## User Modification Area

Users can select the following.

*  Option Switches
      *  ***intel***: Using Support Software with Intel Compilers
      *  ***CLOUDS_ON***: Moisture Profile
      *  ***CLOCK_ON***: Floating Clock on Status Bar
      *  ***INDICIES_ON***: Plot and Include Storm Indicies
      *  ***MAKE_GIFS***: Autocreate Animated GIFs (Only on prepared unix machines)
*  File Organization
      *  ***PATH_TO_WRF_OUTPUT_FILE***: full path and file for "wrfout_d*dd*__*YYYY*-*MM*-*DD*__*hh*:*mm*:*ss*"
      *  ***PATH_TO_TSLIST_FILE***: full path and file for "tslist"
      *  ***PLOT_OUTPUT_DIRECTORY***: full directory path for graphics output
*  Custom Line Colors and Fonts
      *  ***Line_Color***: #Hex code for plot line colors for communication policies


In [2]:
####################################################
####################################################
####################################################

#
# Option Switches
#

intel         = False      

CLOUDS_ON     = True   # Cloud Graph
CLOCK_ON      = True   # Floating Clock
INDICIES_ON   = True   # Plot Storm Indicies
MAKE_GIFS     = True   # Make GIFs (only on UNIX machines)

#
# File Organization
#

PATH_TO_WRF_OUTPUT_FILE  = "/Users/wjc/GitHub/SD_Mines_WRF_REALTIME/ARCHIVE/2024-10-10_00/NETCDF/wrfout_d01_2024-10-10_00.nc"

PATH_TO_TSLIST_FILE      = "/Users/wjc/GitHub/SD_Mines_WRF_REALTIME/namelist_files_and_local_scripts/tslist_short"

PLOT_OUTPUT_DIRECTORY    = "/Users/wjc/GitHub/SD_Mines_WRF_REALTIME/WEB_IMAGES/2024-10-10_00/SKEWTS/"

#
# Custom Colors and Fonts
#

Line_Color = "#002554" # Mines Blue
#Line_Color = "#000000" # Black 

#plt.rcParams['font.family'] = 'OpenSans'

plt.rcParams.update({'text.color'      : Line_Color,
                     'axes.labelcolor' : Line_Color,
					 'axes.edgecolor'  : Line_Color,
					 'xtick.color'     : Line_Color,
					 'ytick.color'     : Line_Color})

#
####################################################
####################################################
####################################################

## Time and Domain Access

Automated Time Extraction for WRF output times, output time steps, and creates the primary time string for output files.

Also in this section, the domain grid extents and surface elevation is extracted.

In [3]:
####################################################
####################################################
####################################################
#
# Time Extraction
#

#
# Crack open NetCDF file 
#

ncf = nc4.Dataset(filename = PATH_TO_WRF_OUTPUT_FILE)

#
# Get Time Information
#

wrf_time_steps = wrf.g_times.get_times(wrfin   =           ncf, 
                                       timeidx = wrf.ALL_TIMES, 
                                       method  =         'cat', 
                                       squeeze =          True, 
                                       cache   =          None, 
                                       meta    =          True, 
                                       _key    =          None)

time_step_minutes = wrf_time_steps[1] - wrf_time_steps[0]
time_step_minutes = time_step_minutes.values.astype('float64')/60e9

nt                = wrf_time_steps.size

simtime_hours     = (nt-1) * time_step_minutes / 60. # Hours

Fdigits           = int( np.log10(simtime_hours *10) )

if (( time_step_minutes % 60 ) == 0):
    FxMINUTES = False
    file_date_string     = wrf_time_steps[0].dt.strftime("%Y-%m-%d_%H").values
    wrf_run_date_string  = wrf_time_steps[0].dt.strftime("%Y-%m-%d %H %Z").values
else:
    FxMINUTES = True
    file_date_string     = wrf_time_steps[0].dt.strftime("%Y-%m-%d_%H%M").values
    wrf_run_date_string  = wrf_time_steps[0].dt.strftime("%Y-%m-%d %H%M %Z").values


#
# Get Latitude and Longitude
#

hgt_sfc = wrf.getvar(wrfin   =       ncf,
                     varname = 'terrain',
                     timeidx =         0, 
                     units   =        'm') * units.m

lat2d, lon2d = wrf.latlon_coords(var = hgt_sfc)

#
# Get Domain ID and Grid Number Extents
#

domain_string = "d"+str(ncf.GRID_ID).zfill(2)

ny = lat2d.sizes.mapping["south_north"]
nx = lat2d.sizes.mapping[ "west_east" ]

#
# Report
#

print("╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")
print("│ ")
print("│ TIME CONTROL")
print("│ ")
print("│     Time Range:", wrf_time_steps[ 0].dt.strftime("%Y-%m-%d_%H%M").values, 
                          "to",
                           wrf_time_steps[-1].dt.strftime("%Y-%m-%d_%H%M").values)
print("│  Time Interval:", time_step_minutes, "minutes")
print("│     Time Steps:", nt)
print("│   Run Duration:", simtime_hours, "hours")
print("│         Domain:", domain_string+";", str(nx)+"we", "X", str(ny)+"sn")
print("│ ")
print("╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")


#
####################################################
####################################################
####################################################

╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ 
│ TIME CONTROL
│ 
│     Time Range: 2024-10-10_0000 to 2024-10-12_0000
│  Time Interval: 15.0 minutes
│     Time Steps: 193
│   Run Duration: 48.0 hours
│         Domain: d01; 149we X 149sn
│ 
╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯


## Read tslist ASCII file

Station extraction uses the tslist format.  

You do *not* need to run WRF with timestep level output to make Skew-Ts with this script!

TSList stations not in a given domain are identified in this step will not be processed.

This section also extracts maximum temperatures, humidity, I & j wrf coordinates and surface elevation.

In [4]:
####################################################
####################################################
####################################################
#
# Read TSLIST Excel File
#

print("╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")
print("│ ")
print("│ Skew-Ts to Plot from TSLIST File")
print("│ ")

print("│ Read File From " + PATH_TO_TSLIST_FILE)

available_time_series_list = pd.read_fwf(filepath_or_buffer = PATH_TO_TSLIST_FILE,
                                         header             =                   2,
                                         colspecs           =           [[ 0,26],
                                                                         [26,30],
                                                                         [30,39],
                                                                         [39,-1]],
                                         names              =    ["Station Name",
                                                                    "Station ID",
                                                                      "Latitude",
                                                                     "Longitude"])

available_time_series_list.set_index(keys = "Station ID", inplace=True)

# available_time_series_list = available_time_series_list.loc[["KBYZ","KOAX","KRAP"]]

#
# Extract I, J WRF Coordinates for each station
#

wrf_x, wrf_y = wrf.ll_to_xy(wrfin     = ncf,
                            latitude  =  available_time_series_list["Latitude"], 
                            longitude = available_time_series_list["Longitude"], 
                            timeidx   = 0, 
                            squeeze   = True, 
                            meta      = False, 
                            stagger   = None, 
                            as_int    = True)

available_time_series_list["wrf_x"] = wrf_x
available_time_series_list["wrf_y"] = wrf_y

available_time_series_list = available_time_series_list[available_time_series_list["wrf_x"] > 0]
available_time_series_list = available_time_series_list[available_time_series_list["wrf_y"] > 0]

available_time_series_list = available_time_series_list[available_time_series_list["wrf_x"] < nx]
available_time_series_list = available_time_series_list[available_time_series_list["wrf_y"] < ny]


del wrf_x, wrf_y

#
# Extract Temperature Range
#

t_4d  = wrf.getvar(wrfin        =           ncf,
                   varname      =          'tc',
                   timeidx      = wrf.ALL_TIMES)     

max_t = []
h_sfc = []
for index, row in available_time_series_list.iterrows():
    max_t.append(   t_4d[:, :,row["wrf_y"], 
                              row["wrf_x"]].max().values)
    h_sfc.append(hgt_sfc[     row["wrf_y"], 
                              row["wrf_x"]].values)

available_time_series_list["h_sfc"] = (h_sfc)  # deg C
available_time_series_list["max_t"] = (max_t)  # deg C

#
# Extract Cloud Data (max q per station)
#

if (CLOUDS_ON):
    qv_4d = wrf.getvar(wrfin        =           ncf,
                       varname      =      'QVAPOR',
                       timeidx      = wrf.ALL_TIMES)     
    
    max_q = []
    for index, row in available_time_series_list.iterrows():
        max_q.append(qv_4d[:, :, row["wrf_y"], 
                                 row["wrf_x"]].max().values * 1000.)

    available_time_series_list["max_q"] = max_q  # g / kg
    
    del max_q

#
# Report
#

display(available_time_series_list)

print("│ ")
print("╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")

#
####################################################
####################################################
####################################################

╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ 
│ Skew-Ts to Plot from TSLIST File
│ 
│ Read File From /Users/wjc/GitHub/SD_Mines_WRF_REALTIME/namelist_files_and_local_scripts/tslist_short


,Station Name,Latitude,Longitude,wrf_x,wrf_y,h_sfc,max_t,max_q
Station ID,,,,,,,,
KTOP,"Topeka NWS, KS",39.0688,-95.6224,143,10,294.29742,29.720917,9.972094


│ 
╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯


## Plot the Skew-T for All Available Stations and All Time Steps

<div class="alert alert-block alert-warning">
    This part of the code is a memory hog at present.  I am working to parallelize it.</a>
</div>

The following fields will be extracted from WRF output for the SkewT plot and thermodynamic and storm indices.  WRF Extraction names in parenthesis are derived using [wrf.getvar()](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html).  

| WRF Extraction Name | Standard Name                  | Units |
|:-------------------:|:-------------------------------:|:-----:|
|         (tc)        | air_temperature                 |  degC |
|         (td)        | dew_point_temperature           |  degC | 
|        (td2m)       | [2-m agl] dew_point_temperature |  degC |
|         (T2)        | [2-m agl] air_temperature       |  degC |
|        (pres)       | air_pressure                    |  hPa  |
|        (PSFC)       | [surface] air-pressure          |  hPa  |
|        (uvmet)      | eastward_wind & northward_wind  | knots |
|       (height)      | geopotential_height             |   m   |
|     (height_agl)    | height                          |   m   |


The following fields will be extracted if a cloud profile is requested.

| WRF Extraction Name | Standard Name                                     |  Units  |
|:-------------------:|:-------------------------------------------------:|:-------:|
|       QVAPOR        | humidity_mixing_ratio                             | kg kg⁻¹ |
|       QCLOUD        | cloud_liquid_water_mixing_ratio                   | kg kg⁻¹ |
|        QICE         | cloud_ice_mixing_ratio                            | kg kg⁻¹ |
|       QRAIN         | mass_concentration_of_liquid_precipitation_in_air | kg kg⁻¹ |
|       QSNOW         | mass_concentration_of_snow_in_air                 | kg kg⁻¹ |
|       QGRAUP        | mass_concentration_of_graupel_in_air              | kg kg⁻¹ |

## Order of Operations

1)   **Outer Time Loop**
        1)  Calculate "UTC-level" time fields (e.g., Model Run, Forecast Hour, etc.
        2)  Pull WRF Fields for the current time step.
        3)  **Inner TSLIST Station Loop**
            1)  Extract Local Station Information from TSLIST DataFrame
            2)  Calculate "Local Timezone-Level" Time Fields (e.g., local time zone by longitude/latitude, local model valid times, floating clock hrs:mins:secs.
            3)  *Data Colation from WRF 3-D & 2-D fields.*
                1)  Construct Sounding/Cloud DataFrame
                2)  Construct Cloud Profile DataFrame
                3)  Construct Hodograph DataFrame
                4)  Construct Isobaric/Geopotential Height Labels DataFrame
                5)  Construct Thermodynamic/Convective Index DataFrame
            4)  *Graphic Object (figure) Creation*
                1)  Plot Space Delineation
                2)  Create Skew-T (axes = "skew")  subplot
                3)  Create Cloud Profile (axes = "axclouds") subplot
                4)  Create Hodograph (axes = "axhodo") subplot
                5)  Create "Walking Clock" and Status Bars
                6)  Create Thermodynamic/Convective Index Table
                7)  Save Single Figure to PNG File.

In [5]:
####################################################
####################################################
####################################################
#
# Automated SkewT Generation
#

#
# Rotate through Available Times
#

print("╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")
print("│ ")
print("│ Rendering Skew-T Diagrams")
print("│ ")

for time in range(nt) :

    print("│ ╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")


    ####################################################
    #
    # Time Variables for All Stations
    #

    percent_done = (1.0 * time) / (nt-1.)

    fx_delta     = wrf_time_steps[time].values - wrf_time_steps[0]
    fx_delta     = fx_delta.values.astype('float64')/60e9
    fx_delta_hr  = int(fx_delta / 60)

    if (FxMINUTES):
        valid_time     = pd.to_datetime(wrf_time_steps[time].values).tz_localize(tz="UTC").strftime("%Y-%m-%d %H%M %Z")
        fx_delta_min   = int(fx_delta % 60)
        fx_code_string = "F"+str(fx_delta_hr).zfill(Fdigits)+"h"+str(fx_delta_min).zfill(2)+"m"
        fx_code_label  = "F"+str(fx_delta_hr).zfill(Fdigits)+":"+str(fx_delta_min).zfill(2)

    else:
        valid_time     = pd.to_datetime(wrf_time_steps[time].values).tz_localize(tz="UTC").strftime("%Y-%m-%d %H %Z")
        fx_code_string = "F"+str(fx_delta_hr).zfill(Fdigits)
        fx_code_label  = "F"+str(fx_delta_hr).zfill(Fdigits)
        
    print("│ │", wrf_run_date_string + " " + fx_code_label, 
                 ":", 
                 (percent_done*100), "complete" )
    print("│ │")
    #
    ####################################################

    ####################################################
    #
    # Extract 3D fields for each timestep
    #
    
    temperature_3d     = wrf.getvar(wrfin   =          ncf,
                                    varname =         'tc',
                                    timeidx =         time) * units.degC

    dew_point_3d       = wrf.getvar(wrfin   =          ncf,
                                    varname =         'td',
                                    timeidx =         time, 
                                    units   =       'degC') * units.degC

    isobar_hgt_3d      = wrf.getvar(wrfin   =          ncf,
                                    varname =       'pres',
                                    timeidx =         time, 
                                    units   =        'hPa') * units.hPa

    uwind_3d, vwind_3d = wrf.getvar(wrfin   =          ncf,
                                    varname =      'uvmet',
                                    timeidx =         time, 
                                    units   =         'kt') * units.kt

    height_3d          = wrf.getvar(wrfin   =          ncf,
                                    varname =     'height',
                                    timeidx =         time, 
                                    units   =          'm') * units.m

    height_agl_3d      = wrf.getvar(wrfin   =          ncf,
                                    varname = 'height_agl',
                                    timeidx =         time, 
                                    units   =          'm') * units.m
    
    td2_sfc            = wrf.getvar(wrfin   =          ncf,
                                    varname =         'td2',
                                    timeidx =         time, 
                                    units   =        'degC') * units.degC
    
    pres_sfc            = (wrf.getvar(wrfin   =          ncf,
                                    varname =         'PSFC',
                                    timeidx =          time)/100.) * units.hPa
    
    t2_sfc             = (wrf.getvar(wrfin   =          ncf,
                                    varname =         'T2',
                                    timeidx =         time)-273.17) * units.degC

    fixed_pres_hgt_3d  = wrf.interplevel(field3d    = height_3d, 
                                         vert       = isobar_hgt_3d, 
                                         desiredlev = [ 200, 250,
                                                        300, 350,
                                                        400, 450,
                                                        500, 550,
                                                        600, 650,
                                                        700, 750,
                                                        800, 850,
                                                        900,
                                                       1000])
    
    fixed_pres_agl_3d  = wrf.interplevel(field3d    = height_agl_3d, 
                                         vert       = isobar_hgt_3d, 
                                         desiredlev = [ 200, 250,
                                                        300, 350,
                                                        400, 450,
                                                        500, 550,
                                                        600, 650,
                                                        700, 750,
                                                        800, 850,
                                                        900,
                                                       1000])
    if (CLOUDS_ON):

        qv_3d          = wrf.getvar(wrfin   =          ncf,
                                    varname =     'QVAPOR',
                                    timeidx =         time)   

        qc_3d          = wrf.getvar(wrfin   =          ncf,
                                    varname =     'QCLOUD',
                                    timeidx =         time)    

        qi_3d          = wrf.getvar(wrfin   =          ncf,
                                    varname =       'QICE',
                                    timeidx =         time)    

        qr_3d          = wrf.getvar(wrfin   =          ncf,
                                    varname =      'QRAIN',
                                    timeidx =         time)    

        qs_3d          = wrf.getvar(wrfin   =          ncf,
                                    varname =      'QSNOW',
                                    timeidx =         time)    

        qg_3d          = wrf.getvar(wrfin   =          ncf,
                                    varname =     'QGRAUP',
                                    timeidx =         time)     
        
    #
    ####################################################

    ####################################################
    #
    # Rotate through Available Stations
    #

    for index, row in available_time_series_list.iterrows():
       
        station_id      = index
        grid_domain     = domain_string
        station_name    = row["Station Name"]
        station_lat     = row["Latitude"]
        station_lon     = row["Longitude"]
        clouds_maxx     = row["max_q"]
        wrf_x           = row["wrf_x"]
        wrf_y           = row["wrf_x"]
        max_temperature = row["max_t"] + 5.
        
        above_30        = np.arange(30,51,5)
        max_axis_temp   = above_30[above_30 > max_temperature].min() 
        

    

        
    

        ####################################################
        #
        # Local Time Information
        #

        tf           = tzf.TimezoneFinder()
        tz           = tf.certain_timezone_at(lng=station_lon, lat=station_lat)
        tzabbr       = pytz.timezone(tz)


        if (FxMINUTES):
            local_time = pd.to_datetime(wrf_time_steps[time].values).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H%M %Z")
        else:
            local_time = pd.to_datetime(wrf_time_steps[time].values).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H %Z")
            
        local_time_zone = pd.to_datetime(wrf_time_steps[time].values).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Z")

        print("│ │ ",  station_id+ " : " + station_name)
    
        #
        # Walking Clock Parameters
        #

        dow            = pd.to_datetime(wrf_time_steps[time].values).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%a")
        time_for_clock = pd.to_datetime(wrf_time_steps[time].values).tz_localize(tz="UTC").tz_convert(tz=tz).time()

        hour           = time_for_clock.hour
        minute         = time_for_clock.minute
        second         = time_for_clock.second

        
        if ((hour >= 6) and (hour < 18)):
            Clock_Color = Line_Color
            Clock_BgndC = "white"           
        else:
            Clock_Color = "white"
            Clock_BgndC = Line_Color        

        circle_theta  = np.deg2rad(np.arange(0,360,0.01))
        circle_radius = circle_theta * 0 + 1

        if (hour > 12) :
            hour = hour - 12

        angles_h = 2*np.pi*hour/12+2*np.pi*minute/(12*60)+2*np.pi*second/(12*60*60)
        angles_m =                 2*np.pi*minute/60     +2*np.pi*second/(60*60)

        #
        ####################################################
        
        
        ####################################################        
        ### ### ### ### ### ### ### ### ### ### ### ### ####
        #
        # Data Colation for Skew T
        #



        #################################################
        #       
        # Make Sounding Dataframe
        #
        
        df_sounding = pd.DataFrame({"pressure"    :   isobar_hgt_3d[:, wrf_y, wrf_x],
                                    "temperature" :  temperature_3d[:, wrf_y, wrf_x],
                                    "dewpoint"    :    dew_point_3d[:, wrf_y, wrf_x],
                                    "u_wind"      :        uwind_3d[:, wrf_y, wrf_x],
                                    "v_wind"      :        vwind_3d[:, wrf_y, wrf_x],
                                    "height"      :       height_3d[:, wrf_y, wrf_x],
                                    "agl"         :   height_agl_3d[:, wrf_y, wrf_x]})

        units_df = {"pressure"    :  "hPa",
                    "temperature" : "degC",
                    "dewpoint"    : "degC",
                    "u_wind"      :   "kt",
                    "v_wind"      :   "kt",
                    "height"      :    "m",
                    "agl"         :    "m"}


        df_sounding = pandas_dataframe_to_unit_arrays(df           = df_sounding, 
                                                      column_units =    units_df)
        #
        #################################################
        
        #################################################
        #
        # Make Clouds Data Frame
        #
        
        if (CLOUDS_ON):
            df_clouds = pd.DataFrame({"height"   :     height_3d[:, wrf_y, wrf_x],
                                      "agl"      : height_agl_3d[:, wrf_y, wrf_x],
                                      "pressure" : isobar_hgt_3d[:, wrf_y, wrf_x],
                                      "qv"       :         qv_3d[:, wrf_y, wrf_x]*1000,
                                      "qc"       :         qc_3d[:, wrf_y, wrf_x]*1000,
                                      "qi"       :         qi_3d[:, wrf_y, wrf_x]*1000,
                                      "qr"       :         qr_3d[:, wrf_y, wrf_x]*1000,
                                      "qs"       :         qs_3d[:, wrf_y, wrf_x]*1000,
                                      "qg"       :         qg_3d[:, wrf_y, wrf_x]*1000})
            
            units_df = {"height"      :    "m",
                        "agl"         :    "m",
                        "pressure"    :  "hPa"} #,
                        #"qv"          : "g/kg",
                        #"qc"          : "g/kg",
                        #"qi"          : "g/kg",
                        #"qr"          : "g/kg",
                        #"qs"          : "g/kg",
                        #"qg"          : "g/kg"}
            
            df_clouds = pandas_dataframe_to_unit_arrays(df           = df_clouds, 
                                                        column_units =  units_df)
            
        #
        #################################################

        #################################################
        #
        # Make Hodograph Data Frame
        #

        #
        # Create the working Hodograph Dataframe.
        #

        df_hodo = pd.DataFrame.from_dict(df_sounding)[["agl",
                                                       "u_wind",
                                                       "v_wind"]].copy()
        #
        # Create the working Hodograph Dataframe.
        #

        df_hodo_fixed_hgt = pd.DataFrame({"agl":[   500,
                                                   1000,
                                                   2000,
                                                   3000,
                                                   4000,
                                                   5000,
                                                   6000,
                                                   7000,
                                                   8000,
                                                   9000,
                                                  10000]})

        #
        # Create and apply an Interpolation Object using the Sounding Levels
        #

        fx = scintp.interp1d(x    = df_hodo[   "agl"].values, 
                             y    = df_hodo["u_wind"].values,
                             kind = "cubic")


        fy = scintp.interp1d(x    = df_hodo[   "agl"].values, 
                             y    = df_hodo["v_wind"].values,
                             kind = "cubic")

        df_hodo_fixed_hgt["u_wind"] = fx(df_hodo_fixed_hgt["agl"].values)
        df_hodo_fixed_hgt["v_wind"] = fy(df_hodo_fixed_hgt["agl"].values)

        #
        # Merge, Sort and convert to m s-1
        #

        df_hodo = pd.concat([df_hodo,
                             df_hodo_fixed_hgt]).sort_values(by = "agl",
                                                             ignore_index = True)
        
        df_hodo["speed"] = np.sqrt((df_hodo["u_wind"].values**2) 
                                 + (df_hodo["v_wind"].values**2))       

        df_hodo[df_hodo["speed"] <= 40] # peak at 40 kts ~ 20 m s-1

        units_df = {"u_wind"    : "kts",
                    "v_wind"    : "kts",
                    "speed"     : "kts",
                    "height"    :   "m",
                    "agl"       :   "m"}
        
        df_hodo = pandas_dataframe_to_unit_arrays(df           =  df_hodo,
                                                  column_units = units_df)
    
        df_hodo["u_wind"] = df_hodo["u_wind"].to(units("m/s"))        
        df_hodo["v_wind"] = df_hodo["v_wind"].to(units("m/s"))      
        df_hodo["speed"]  = df_hodo[ "speed"].to(units("m/s"))
                   
        #
        #################################################

        #################################################
        #
        # Make Isobaric Height Data Frame
        #

        df_isobaric_levels           = pd.DataFrame({"pressure":fixed_pres_hgt_3d.coords["level"].values})
        df_isobaric_levels["height"] = np.round(fixed_pres_hgt_3d[:, wrf_y, wrf_x])
        df_isobaric_levels["agl"]    = np.round(fixed_pres_agl_3d[:, wrf_y, wrf_x])

        df_isobaric_levels = df_isobaric_levels[np.isfinite(df_isobaric_levels["height"])]

        df_isobaric_levels = pd.concat([df_isobaric_levels,
                                        pd.DataFrame(data = {"pressure":[pres_sfc[wrf_y, wrf_x].values],
                                                             "height":[np.round(hgt_sfc[wrf_y, wrf_x].values)],
                                                             "agl":[0]})],
                                       ignore_index=True)
        #
        #################################################
        

        #################################################
        #
        # Make Storm Indicies
        #

        if (INDICIES_ON):

            z_700_500_3d = wrf.vinterp(wrfin         =           ncf,
                                       field         = height_agl_3d,
                                       vert_coord    =    "pressure",
                                       interp_levels =     [700,500],
                                       log_p         =          True,
                                       timeidx       =          time) * units.m
    
            t_700_500_3d = wrf.vinterp(wrfin        =             ncf,
                                       field         = temperature_3d,
                                       vert_coord    =     "pressure",
                                       interp_levels =      [700,500],
                                       log_p         =           True,
                                       timeidx       =           time) * units.degC
    
            z_700_500 = z_700_500_3d[:,wrf_y,wrf_x]
            t_700_500 = t_700_500_3d[:,wrf_y,wrf_x]
            
            lcl_pressure, lcl_temperature = mpcalc.lcl(df_sounding["pressure"][0],
                                                       df_sounding["temperature"][0],
                                                       df_sounding["dewpoint"][0])
    
            lfc_pressure, lfc_temperature = mpcalc.lfc(df_sounding["pressure"],
                                                       df_sounding["temperature"],
                                                       df_sounding["dewpoint"])
    
            el_pressure, el_temperature = mpcalc.el(df_sounding["pressure"],
                                                    df_sounding["temperature"],
                                                    df_sounding["dewpoint"])
    
            parcel_profile = mpcalc.parcel_profile(df_sounding["pressure"],
                                                   df_sounding["temperature"][0],
                                                   df_sounding["dewpoint"][0])
    
            surface_cape, surface_cin = mpcalc.surface_based_cape_cin(df_sounding["pressure"],
                                                                      df_sounding["temperature"],
                                                                      df_sounding["dewpoint"])
    
            lcl_hgt = np.round(mpcalc.pressure_to_height_std(lcl_pressure), 
                               decimals=3).to(units.meter)
            
            lfc_hgt = np.round(mpcalc.pressure_to_height_std(lfc_pressure), 
                               decimals=3).to(units.meter)
    
            sb_cape, sb_cin = mpcalc.surface_based_cape_cin(df_sounding["pressure"], 
                                                            df_sounding["temperature"], 
                                                            df_sounding["dewpoint"])
            
            ml_cape, ml_cin = mpcalc.mixed_layer_cape_cin(df_sounding["pressure"], 
                                                          df_sounding["temperature"], 
                                                          df_sounding["dewpoint"])
            
            mu_cape, mu_cin = mpcalc.most_unstable_cape_cin(df_sounding["pressure"], 
                                                            df_sounding["temperature"], 
                                                            df_sounding["dewpoint"])
    
            t_700m500  =  (t_700_500[1].values- t_700_500[0].values)*units.delta_degC
            z_700m500  = ((z_700_500[1].values- z_700_500[0].values)/1000)*units.km
            lr_700_500 = np.round(-1 * np.divide( t_700m500, z_700m500),2)
    
            sbcape = np.round(sb_cape, 1)
            sbcin  = np.round(sb_cin,  1)
            mlcape = np.round(ml_cape, 1)
            mlcin  = np.round(ml_cin,  1)
            mucape = np.round(mu_cape, 1)
    
            u_shear01, v_shear01 = mpcalc.bulk_shear(df_sounding["pressure"], 
                                                     df_sounding["u_wind"].to(units.meter/units.second), 
                                                     df_sounding["v_wind"].to(units.meter/units.second), 
                                                     depth = 1000 * units.meter)
            
            shear01 = np.round((np.sqrt(u_shear01**2 + v_shear01**2)), 1)
            
            u_shear06, v_shear06 = mpcalc.bulk_shear(df_sounding["pressure"], 
                                                     df_sounding["u_wind"].to(units.meter/units.second), 
                                                     df_sounding["v_wind"].to(units.meter/units.second), 
                                                     depth = 6000 * units.meter)
            
            shear06 = np.round((np.sqrt(u_shear06**2 + v_shear06**2)), 1)
            
            rmover, lmover, mean = mpcalc.bunkers_storm_motion(df_sounding["pressure"], 
                                                               df_sounding["u_wind"].to(units.meter/units.second), 
                                                               df_sounding["v_wind"].to(units.meter/units.second), 
                                                               df_sounding["agl"])
            
            srh_01_pos, srh_01_neg, srh_01_tot = mpcalc.storm_relative_helicity(df_sounding["height"],
                                                                                df_sounding["u_wind"].to(units.meter/units.second), 
                                                                                df_sounding["v_wind"].to(units.meter/units.second),  
                                                                                depth   = 1000 * units.meter, 
                                                                                bottom  = df_sounding["agl"][0], 
                                                                                storm_u = lmover[0], 
                                                                                storm_v = lmover[1])
            srh_01 = np.round(srh_01_neg, 1)
            
            srh_03_pos, srh_03_neg, srh_03_tot = mpcalc.storm_relative_helicity(df_sounding["height"],
                                                                                df_sounding["u_wind"].to(units.meter/units.second), 
                                                                                df_sounding["v_wind"].to(units.meter/units.second), 
                                                                                depth   = 3000 * units.meter, 
                                                                                bottom  = df_sounding["agl"][0], 
                                                                                storm_u = lmover[0], storm_v = lmover[1])
            srh_03 = np.round(srh_03_neg, 1)

        #
        #################################################

        #
        # End Data Colation for Skew T
        #
        ### ### ### ### ### ### ### ### ### ### ### ### ####
        ####################################################

        
        ####################################################
        ### ### ### ### ### ### ### ### ### ### ### ### ####
        #
        # Render SkewT Plot
        #
        
        ###################################################
        #
        # Generate SkewT
        #

        fig = plt.figure(figsize   = (9, 9))

        fig.suptitle(station_name + "; " + wrf_run_date_string + " " + fx_code_label + "; WRF Domain " + domain_string, 
                     fontsize=18,
                     verticalalignment = "center",
                     horizontalalignment = "center")

        # Grid for plots

        gs   = gridspec.GridSpec(nrows = 3, 
                                 ncols = 3)

        skew = SkewT(fig      =       fig, 
                     rotation =        45,  
                     aspect   = (1.6*(max_axis_temp + 25) - 7.5),
                     rect     = [0.10, 0.00, 0.60, 0.95])
        
        skew.ax.set_xlim(-25, max_axis_temp )

        # Plot the sounding using normal plotting functions, in this case using
        # log scaling in Y, as dictated by the typical meteorological plot

        skew.plot(df_sounding["pressure"], 
                  df_sounding["temperature"], 
                  color = 'red')
        
        skew.plot(df_sounding["pressure"], 
                  df_sounding["dewpoint"], 
                  color = 'green')
        
        skew.plot(df_sounding["pressure"], 
                  parcel_profile, 
                  color = Line_Color, 
                  linewidth = 0.5)

        # Mask barbs to be below 100 hPa only

        mask = (df_sounding["pressure"] >= 100 * units.hPa)
        for i in range(mask.size):
            if (df_sounding["pressure"][i] > 500 * units.hPa) :
                if (i%2 == 1):
                    mask[i] = False

        skew.plot_barbs(df_sounding["pressure"][mask], 
                        df_sounding[  "u_wind"][mask].astype('float64'), 
                        df_sounding[  "v_wind"][mask].astype('float64'),
                        color=Line_Color)

        # Add the relevant special lines

        skew.plot_dry_adiabats(  linewidths = 0.75)
        skew.plot_moist_adiabats(linewidths = 0.75)
        skew.plot_mixing_lines(  linewidths = 0.75)

        #
        ###################################################


        ###################################################
        #
        # Skew-T Thermodynamics
        #


        # Shade areas

        skew.shade_cin(df_sounding["pressure"], 
                       df_sounding["temperature"], 
                       parcel_profile, 
                       color = "lightcyan")
        
        skew.shade_cape(df_sounding["pressure"], 
                        df_sounding["temperature"], 
                        parcel_profile, 
                        color = "mistyrose")


        # Good bounds for aspect ratio

        
        skew.ax.set_title(valid_time + "  (" + local_time+")", fontsize=15)


        if lcl_pressure:
            skew.ax.plot(lcl_temperature, 
                         lcl_pressure, 
                         marker          =     "_", 
                         color           = 'orange', 
                         markersize      =      30, 
                         markeredgewidth =       3)

        if lfc_pressure:
            skew.ax.plot(lfc_temperature, 
                         lfc_pressure, 
                         marker          =     "_", 
                         color           = 'brown', 
                         markersize      =      30, 
                         markeredgewidth =       3)

        if el_pressure:
            skew.ax.plot(el_temperature, 
                         el_pressure, 
                         marker          =    "_", 
                         color           = 'blue', 
                         markersize      =     30, 
                         markeredgewidth =      3)
            
        skew.ax.spines["right"].set_visible(False)
        skew.ax.spines[  "top"].set_color("white")
        skew.ax.set_xlabel('Temperature (\N{DEGREE CELSIUS})')
        skew.ax.set_ylabel("Isobaric Height (hPa)")

        skew_box          = skew.ax.get_position()
        skew_box_x_start  = skew_box.x0
        skew_box_y_start  = skew_box.y0
        skew_box_x_end    = skew_box.x1
        skew_box_y_end    = skew_box.y1
        skew_box_x_length = skew_box.x1 - skew_box.x0
        skew_box_y_length = skew_box.y1 - skew_box.y0
    

        #
        ###################################################


        
        ###################################################
        #
        # Create a Cloud Water Profile
        #
     
        if (CLOUDS_ON):
            
            axclouds = SkewT(fig      =       fig,
                             rotation =         0, 
                             aspect   =     'auto',
                             rect     = [0.77, skew_box_y_start, 0.28, skew_box_y_length])

            axclouds.ax.xaxis.set_units(("gram / kilogram"))
            axclouds.ax.yaxis.set_units("hectopascal")

            axclouds.ax.set_title('Hodograph & Moisture Profile', fontsize=15)
            axclouds.ax.set_xlabel('Mixing Ratio (g/kg)')
            axclouds.ax.set_ylabel("")
            axclouds.ax.set_xlim(0, np.ceil(clouds_maxx))
            axclouds.ax.xaxis.set_major_locator(MultipleLocator(1))
            
            axclouds.ax.spines["right"].set_visible(False)
            axclouds.ax.spines["left"].set_visible(False)
            axclouds.ax.spines[  "top"].set_color("white")
            

            cloud_legend = ["$q_v$"]
            axclouds.plot(    df_clouds["pressure"], df_clouds["qv"], color="greenyellow")
            if np.any(df_clouds["qc"] > 0):
                axclouds.plot(df_clouds["pressure"], df_clouds["qc"], color="darkgrey") 
                cloud_legend.append("$q_c$")
            if np.any(df_clouds["qi"] > 0):
                axclouds.plot(df_clouds["pressure"], df_clouds["qi"], color="cyan")            
                cloud_legend.append("$q_i$")
            if np.any(df_clouds["qr"] > 0):
                axclouds.plot(df_clouds["pressure"], df_clouds["qr"], color="darkgreen")            
                cloud_legend.append("$q_r$")
            if np.any(df_clouds["qs"] > 0):
                axclouds.plot(df_clouds["pressure"], df_clouds["qs"], color="blue")            
                cloud_legend.append("$q_s$")
            if np.any(df_clouds["qg"] > 0):
                axclouds.plot(df_clouds["pressure"], df_clouds["qg"], color="red")  
                cloud_legend.append("$q_g$")


            axclouds.ax.legend(cloud_legend,
                               loc     = 'center right',
                               frameon = False)
        
            axclouds_box          = axclouds.ax.get_position()
            axclouds_box_x_start  = axclouds_box.x0
            axclouds_box_y_start  = axclouds_box.y0
            axclouds_box_x_end    = axclouds_box.x1
            axclouds_box_y_end    = axclouds_box.y1
            axclouds_box_x_length = axclouds_box.x1 - axclouds_box.x0
            axclouds_box_y_length = axclouds_box.y1 - axclouds_box.y0


        #
        ###################################################             

    
        ###################################################
        #
        # Create a hodograph
        #
        
        do_hodo = True
        
        if (do_hodo):
            
            mask = df_hodo["agl"] <= 10 * units.km

            axhodo = fig.add_axes(rect   = [0.75, 
                                            skew_box_y_start+skew_box_y_length-0.32, 
                                            0.3, 
                                            0.3],
                                  zorder =  1001)
            axhodo.zorder=1001

            h = Hodograph(ax              = axhodo, 
                          component_range = 40.)

            h.add_grid(increment=10)
            axhodo.spines["right"].set_visible(False)
            axhodo.spines[  "top"].set_visible(False)


            cmh = h.plot_colormapped(u    = df_hodo["u_wind"][mask], 
                                     v    = df_hodo["v_wind"][mask], 
                                     c    = df_hodo["agl"][mask].to(units.km), 
                                     cmap = "jet")
            cmh.set_clim(0, 10)
            

            fig.patches.extend([plt.Rectangle((0.73,skew_box_y_start+skew_box_y_length-0.35),0.3,0.355,
                              fill=True, color='w', alpha=0.99, zorder=1000,
                              transform=fig.transFigure, figure=fig)])

            cbhodo = plt.colorbar(mappable = cmh, 
                                  ax = axhodo,
                                  #labelpad = -2,
                                  orientation = "horizontal", 
                                  label       = 'Height (km AGL)')
                    
            cbhodo.ax.zorder = 1001

            axhodo.set_xlabel("u (m s$^{-1}$)",labelpad=-2)
            axhodo.set_ylabel("v (m s$^{-1}$)",labelpad=-2)
        #
        ###################################################  
           
        
        ###################################################
        #
        # Add Status Bars 
        #
        
        rect1 = patches.Rectangle(xy        = (0, 0-0.01/2),
                                  width     = percent_done,
                                  height    = 0.01, 
                                  edgecolor = Line_Color, 
                                  facecolor = Line_Color,
                                  transform = skew.ax.transAxes)
        skew.ax.add_patch(rect1)

        rect2 = patches.Rectangle(xy        = (0, 0-0.01/2),
                                  width     = percent_done,
                                  height    = 0.015, 
                                  edgecolor = Line_Color, 
                                  facecolor = Line_Color,
                                  transform = axhodo.transAxes, zorder = 1001)
        axhodo.add_patch(rect2)

        if (CLOUDS_ON):
            rect3 = patches.Rectangle(xy        = (0, 0-0.01/2),
                                      width     = percent_done,
                                      height    = 0.01, 
                                      edgecolor = Line_Color, 
                                      facecolor = Line_Color,
                                      transform = axclouds.ax.transAxes)
            axclouds.ax.add_patch(rect3)
        
        #######
        #
        # Walking Clock
        #
        #######
        
        plot_box          = skew.ax.get_position()
        plot_box_x_start  = plot_box.x0
        plot_box_y_start  = plot_box.y0
        plot_box_x_end    = plot_box.x1
        plot_box_y_end    = plot_box.y1
        plot_box_x_length = plot_box.x1 - plot_box.x0
        plot_box_y_length = plot_box.y1 - plot_box.y0

        size_of_clock = 0.05

        x_clock = percent_done*(plot_box_x_end-plot_box_x_start) + plot_box_x_start - size_of_clock/2

        y_clock = plot_box_y_start-size_of_clock/2
        
        x_dow = percent_done
        y_dow = size_of_clock    
        
        skew.ax.annotate(dow+"-"+local_time_zone, 
                         [x_dow,y_dow],
                         horizontalalignment = "center",
                         verticalalignment   = "center",
                         xycoords            = 'axes fraction')


        
        axins = fig.add_axes(rect     =    [x_clock,
                                            y_clock,
                                            size_of_clock,
                                            size_of_clock],
                             projection =  "polar")
        
        plt.setp(axins.get_yticklabels(), visible=False)
        plt.setp(axins.get_xticklabels(), visible=False)
        axins.spines['polar'].set_visible(False)
        axins.set_ylim(0,1)
        axins.set_theta_zero_location('N')
        axins.set_theta_direction(-1)
        axins.set_facecolor(Clock_BgndC)
        axins.grid(False)

        axins.plot([angles_h,angles_h], [0,0.60], color=Clock_Color, linewidth=1)
        axins.plot([angles_m,angles_m], [0,0.95], color=Clock_Color, linewidth=1)
        axins.plot(circle_theta, circle_radius,   color="darkgrey",  linewidth=1)

        #
        ###################################################

        ###################################################
        #
        # Add Descriptive Statistics
        #   

        if (INDICIES_ON): 
        
            xtext_start =  0.22;  0.115
            ytext_start =  0.9
            text_deltay =  0.02
            xtext_numbr =  xtext_start + 0.005
    
            
            plt.figtext( xtext_start, ytext_start - text_deltay* 0, 'LCL Height:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 0, '{0:.1f} m'.format(lcl_hgt.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay* 1, 'LFC Height:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 1, '{0:.1f} m'.format(lfc_hgt.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay* 2, 'MLLR:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 2, '{0:.1f} K'.format(lr_700_500.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay* 3, 'SBCAPE:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 3, '{0:.1f} J/kg'.format(sbcape.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay* 4, 'SBCIN:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 4, '{0:.1f} J/kg'.format(sbcin.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay* 5, 'MLCAPE:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 5, '{0:.1f} J/kg'.format(mlcape.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay* 6, 'MLCIN:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 6, '{0:.1f} J/kg'.format(mlcin.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay* 7, 'MUCAPE:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 7, '{0:.1f} J/kg'.format(mucape.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay* 8, 'Shear 0-1 km:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 8, '{0:.1f} m/s'.format(shear01.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay* 9, 'Shear 0-6 km:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay* 9, '{0:.1f} m/s'.format(shear06.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay*10, 'SRH 0-1 km:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay*10, '{0:.1f} m\u00b2/s\u00b2'.format(srh_01.magnitude))
            plt.figtext( xtext_start, ytext_start - text_deltay*11, 'SRH 0-3 km:', horizontalalignment = "right")
            plt.figtext( xtext_numbr, ytext_start - text_deltay*11, '{0:.1f} m\u00b2/s\u00b2'.format(srh_03.magnitude))

        #
        ###################################################           


        ###################################################
        #
        # Close SkewT and Save to File
        #

        #
        # Creating Graphics And File
        #

        sounding_file_name_png = ("wrfout_"  \
                                  + domain_string \
                                  + "_"           \
                                  + file_date_string \
                                  + "_" + fx_code_string \
                                  + "_SKEWT_" \
                                  + station_id \
                                  + ".png")

        graphics_directory = (PLOT_OUTPUT_DIRECTORY 
                              + domain_string        
                              + "/"                 
                              + station_id          
                              + "/"  )  
    
        try:
            os.makedirs(graphics_directory)
            print("│ │   Creating " + graphics_directory)
        except FileExistsError:
            pass
            
        #
        # "Bag it!"
        #
        
        plt.savefig(graphics_directory + sounding_file_name_png,
                    facecolor   = 'white', 
                    transparent =   False,
                    bbox_inches = 'tight', 
                    pad_inches  =       0)
                    
        plt.close('all')
        
    
        #
        ###################################################

        #
        # End Rendering of Skew T
        #
        ### ### ### ### ### ### ### ### ### ### ### ### ####
        ####################################################
    
    # 
    # End Station Loop
    #
    ####################################################

    print("│ ╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")


# 
# End Time Loop
#
####################################################    

print("│ ")
print("╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")

#
####################################################
####################################################
####################################################

╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ 
│ Rendering Skew-T Diagrams
│ 
│ ╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ │ 2024-10-10 0000  F00:00 : 0.0 complete
│ │
│ │  KTOP : Topeka NWS, KS
│ │   Creating /Users/wjc/GitHub/SD_Mines_WRF_REALTIME/WEB_IMAGES/2024-10-10_00/SKEWTS/d01/KTOP/
│ ╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ ╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ │ 2024-10-10 0000  F00:15 : 0.5208333333333333 complete
│ │
│ │  KTOP : Topeka NWS, KS
│ ╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ ╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ │ 2024-10-10 0000  F00:30 : 1.0416666666666665 complete
│ │
│ │  KTOP : Topeka NWS, KS
│ ╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ ╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ │ 2024-10-10 0000  F00:45 : 1.5625 complete
│ │
│ │  KTOP : Topeka NWS, KS
│ ╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ ╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ │ 2024-10-10 0000  F01:00 : 2.083333333333333 complete
│ │
│ │  KTOP : Topeka NWS, KS
│ ╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ ╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ │ 2024-10-10 0000  F01:15 : 2.604166666666667 complete
│ │
│ │  KTOP : Topeka NWS, KS
│ ╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ ╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ │ 2024-10-10 0000  F01:30 : 

## Creating Animated GIFS

This option is only available on prepared unix systems with Imagemagick installed.


In [6]:
####################################################
#
# making gifs Unix Machines Only
#

if (MAKE_GIFS): 

    print("╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")
    print("│ ")
    print("│ Rendering Skew-T Diagrams")
    print("│ ")


    for index, row in available_time_series_list.iterrows():
   
        station_id      = index
        grid_domain     = domain_string
    
        magick_command = "magick "
    
    
        sounding_file_name_png = ("wrfout_"  \
                                  + domain_string \
                                  + "_"           \
                                  + file_date_string \
                                  + "_F*" \
                                  + "_SKEWT_" \
                                  + station_id \
                                  + ".png")
    
        sounding_file_name_gif = ("wrfout_"  \
                                  + domain_string \
                                  + "_"           \
                                  + file_date_string \
                                  + "_SKEWT_" \
                                  + station_id \
                                  + ".gif")
    
        sounding_script_name   = ("wrfout_"  \
                                  + domain_string \
                                  + "_"           \
                                  + file_date_string \
                                  + "_SKEWT_" \
                                  + station_id \
                                  + "_gif.sh")
    
        graphics_directory = (PLOT_OUTPUT_DIRECTORY 
                              + domain_string        
                              + "/"                 
                              + station_id          
                              + "/"  )  
    
        print("│ creating " + sounding_file_name_gif)
        os.system(magick_command + " -delay 10 " 
                      + graphics_directory + sounding_file_name_png 
                      + " " 
                      + graphics_directory + sounding_file_name_gif) 

        print()

    print("│ ")
    print("╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")

╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ 
│ Rendering Skew-T Diagrams
│ 
│ creating wrfout_d01_2024-10-10_0000_SKEWT_KTOP.gif

│ 
╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯


## Ending ScriptLine_Color

In [7]:
####################################################
####################################################
####################################################
#
# End of Script
#

print("╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")
print("│ ")
print("│ End Sounding Plotting Script")
print("│ ")
print("╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯")

#
####################################################
####################################################
####################################################

╭⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
│ 
│ End Sounding Plotting Script
│ 
╰⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
